# Problem Set 3

### Lukibanov Ilya

## Lalonde NSW Data

In [1]:
from causalinference import CausalModel
from causalinference.utils import lalonde_data

#### A

Load data and make summary statistics

In [2]:
Y, D, X = lalonde_data()
causal = CausalModel(Y, D, X)
print(causal.summary_stats)


Summary Statistics

                       Controls (N_c=260)         Treated (N_t=185)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y        4.555        5.484        6.349        7.867        1.794

                       Controls (N_c=260)         Treated (N_t=185)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0        0.827        0.379        0.843        0.365        0.044
             X1        0.108        0.311        0.059        0.237       -0.175
             X2       25.054        7.058       25.816        7.155        0.107
             X3        0.154        0.361        0.189        0.393        0.094
             X4        0.835        0.372        0.708        0.456       -0.304
      

Nodegree has the highest difference

#### B

In [3]:
causal.est_propensity_s(lin_B=[6,7,8,9])
print (causal.propensity)


Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept     -3.480      4.471     -0.778      0.436    -12.243      5.283
            X6      0.034      0.051      0.667      0.505     -0.066      0.133
            X7     -0.236      0.386     -0.611      0.541     -0.992      0.521
            X8      0.058      0.051      1.144      0.253     -0.041      0.158
            X9     -3.477      1.652     -2.104      0.035     -6.716     -0.238
            X4      7.329      4.255      1.723      0.085     -1.010     15.668
            X1     -0.653      0.385     -1.696      0.090     -1.409      0.102
            X5      0.290      0.370      0.783      0.433     -0.435      1.015
         X4*X5     -0.668      0.349     -1.915      0.056     -1.352      0.016
         X6*X4     -0.130      0.057     -2.286      0.022     -0.

Algorithm also selected Nodegree, Hispanic, Education, Nodegree x  Education, Nodegree x Earnings74, and Education x Unemployment75

#### C

In [4]:
causal.trim_s()
print("Selected cutoff:", causal.cutoff)
print (causal.summary_stats)

Selected cutoff: 0.131042280162

Summary Statistics

                       Controls (N_c=256)         Treated (N_t=182)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y        4.543        5.501        6.237        7.587        1.694

                       Controls (N_c=256)         Treated (N_t=182)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0        0.828        0.378        0.841        0.367        0.034
             X1        0.109        0.313        0.060        0.239       -0.176
             X2       25.074        7.091       25.841        7.208        0.107
             X3        0.156        0.364        0.187        0.391        0.081
             X4        0.832        0.375        0.714 

There were dropped 260-256=4 control observations and 185-182=3 treated observations.

#### D

In [5]:
causal.stratify_s()
print(causal.strata)


Stratification Summary

              Propensity Score         Sample Size     Ave. Propensity   Outcome
   Stratum      Min.      Max.  Controls   Treated  Controls   Treated  Raw-diff
--------------------------------------------------------------------------------
         1     0.131     0.379       153        67     0.327     0.332     0.788
         2     0.380     0.483        69        63     0.435     0.443     1.587
         3     0.487     0.852        34        52     0.596     0.619     3.044



There are three bins.

#### E

In [6]:
causal.est_via_ols()
causal.est_via_blocking()
causal.est_via_matching(matches=2, bias_adj=True)
print(causal.estimates)


Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      1.467      0.638      2.299      0.022      0.216      2.718
           ATC      1.385      0.652      2.123      0.034      0.106      2.663
           ATT      1.583      0.651      2.432      0.015      0.307      2.858

Treatment Effect Estimates: Blocking

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      1.542      0.641      2.406      0.016      0.286      2.798
           ATC      1.402      0.654      2.145      0.032      0.121      2.683
           ATT      1.739      0.663      2.623      0.009      0.440      3.039

Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int

Estimates do not differ a lot: only around 10% between different estimators.

## Document Classification

Import libraries

In [7]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import linear_model
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from pprint import pprint
import numpy as np

#### A

get the data for train and test

In [8]:
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

Topic names:

In [9]:
print(list(newsgroups_train.target_names))

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


Sample posts:

In [10]:
pprint(newsgroups_train.data[110:114])

['From: cramer@optilink.COM (Clayton Cramer)\n'
 'Subject: Re: New Study Out On Gay Percentage\n'
 'Organization: Optilink Corporation, Petaluma, CA\n'
 'Lines: 19\n'
 '\n'
 'In article <C5L0v1.JCv@news.cso.uiuc.edu>, dans@uxa.cso.uiuc.edu (Dan S.) '
 'writes:\n'
 "> Don't forget about the culture.  Sadly, we don't (as a society) look upon\n"
 '> homosexuality as normal (and as we are all too well aware, there are alot\n'
 '> of people who condemn it).  As a result, the gay population is not '
 'encouraged\n'
 '> to develop "non-promiscuous" relationships.  In fact there are many '
 'roadblocks\n'
 '> put in the way of such committed relationships.  It is as if the '
 'heterosexual\n'
 '\n'
 "Such as?  Not being able to get married isn't a roadblock to a permanent\n"
 "relationship.  Lack of a marriage certificate doesn't force a couple\n"
 'to break up.  This is an excuse used by homosexuals because the \n'
 'alternative is to ask why they are so much more promiscuous than \n'
 'strai

#### B

Converting into bags of words (creating dictionary from train data):

In [11]:
c_vec = CountVectorizer()
c_vec.fit(newsgroups_train.data)
text_data_num = c_vec.transform(newsgroups_train.data)
text_data_test = c_vec.transform(newsgroups_test.data)
text_data_num.shape

(11314, 130107)

So, dictionary has the lenght of 130107 words

#### C

Use SVD to reduce number of dimensions:

In [12]:
K = 30
svd = TruncatedSVD(n_components = K)
svd.fit(text_data_num)
transformed_data = svd.transform(text_data_num)
transrormed_test = svd.transform(text_data_test)
svd.explained_variance_ratio_.sum()

0.98811968477911039

In [13]:
transformed_data.shape

(11314, 30)

#### D

We cannot use Multinomial Naive Bayes because it requires only positive numbers (and during dimensionality reduction some of the new variable will be negative); however, it has a great performance without dimensionality reduction:

In [14]:
mnb = MultinomialNB().fit(text_data_num, newsgroups_train.target)
print(metrics.accuracy_score(newsgroups_train.target, mnb.predict(text_data_num)))

0.924518295917


In [15]:
print(metrics.accuracy_score(newsgroups_test.target, mnb.predict(text_data_test)))

0.772835900159


In-sample 92% are right classified, and 77% out of sample.

Let's use favourite tool of empirical IO and Marketing - multinomial logit:

In [16]:
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(transformed_data, newsgroups_train.target)

In [17]:
print(metrics.accuracy_score(newsgroups_train.target, mul_lr.predict(transformed_data)))

0.403482411172


In [18]:
print(metrics.accuracy_score(newsgroups_test.target, mul_lr.predict(transrormed_test)))

0.353823685608


40% in-sample - not so good, but 35% out of sample - quite impressive

Let's try Gaussian naive bayes:

In [19]:
gnb = GaussianNB().fit(transformed_data, newsgroups_train.target)

In [20]:
print(metrics.accuracy_score(newsgroups_train.target, gnb.predict(transformed_data)))

0.129397207


In [21]:
print(metrics.accuracy_score(newsgroups_test.target, gnb.predict(transrormed_test)))

0.128518321827


Unimpressive performance, but almost no difference between in and out of sample

#### E

In [22]:
k = 0
results = np.zeros((10,2))
results[k,0] = metrics.accuracy_score(newsgroups_train.target, mul_lr.predict(transformed_data))
results[k,1] = metrics.accuracy_score(newsgroups_test.target, mul_lr.predict(transrormed_test))

In [23]:
for i in [40,50,60,70,80,90,100]:
    k = k + 1
    svd = TruncatedSVD(n_components = i)
    svd.fit(text_data_num)
    transformed_data = svd.transform(text_data_num)
    transrormed_test = svd.transform(text_data_test)
    mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(transformed_data, newsgroups_train.target)
    print(i)
    results[k,0] = metrics.accuracy_score(newsgroups_train.target, mul_lr.predict(transformed_data))
    results[k,1] = metrics.accuracy_score(newsgroups_test.target, mul_lr.predict(transrormed_test))
    print(results[k,0])
    print(results[k,1])

40
0.483825349125
0.41409984068
50
0.526073890755
0.450876261285
60
0.560809616404
0.482873074881
70
0.592186671381
0.509691980882
80
0.635142301573
0.538767923526
90
0.663249071946
0.566117896973
100
0.690737139827
0.583244822092


Accuracy is increasing with number of dimensions. If we use multinomial logit for the whole sample, we would get:

In [24]:
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(text_data_num, newsgroups_train.target)

In [25]:
print(metrics.accuracy_score(newsgroups_train.target, mul_lr.predict(text_data_num)))

0.99991161393


In [26]:
print(metrics.accuracy_score(newsgroups_test.target, mul_lr.predict(text_data_test)))

0.789298990972


So, out of sample accuracy is 78.9% which is even higher than MultinomialNB. However, it takes a lot of time to compute. The best K is the whole sample.